How to choose foods that satisfy nutritional requirements while minimizing costs or maximizing satiety?

In [1]:
from collections import namedtuple
from docplex.mp.model import Model
from docplex.util.environment import get_environment

In [3]:
FOODS = [
    ("Roasted Chicken", 0.84, 0, 10),
    ("Spaghetti W/ Sauce", 0.78, 0, 10),
    ("Tomato,Red,Ripe,Raw", 0.27, 0, 10),
    ("Apple,Raw,W/Skin", .24, 0, 10),
    ("Grapes", 0.32, 0, 10),
    ("Chocolate Chip Cookies", 0.03, 0, 10),
    ("Lowfat Milk", 0.23, 0, 10),
    ("Raisin Brn", 0.34, 0, 10),
    ("Hotdog", 0.31, 0, 10)
]

NUTRIENTS = [
    ("Calories", 2000, 2500),
    ("Calcium", 800, 1600),
    ("Iron", 10, 30),
    ("Vit_A", 5000, 50000),
    ("Dietary_Fiber", 25, 100),
    ("Carbohydrates", 0, 300),
    ("Protein", 50, 100)
]

FOOD_NUTRIENTS = [
    ("Roasted Chicken", 277.4, 21.9, 1.8, 77.4, 0, 0, 42.2),
    ("Spaghetti W/ Sauce", 358.2, 80.2, 2.3, 3055.2, 11.6, 58.3, 8.2),
    ("Tomato,Red,Ripe,Raw", 25.8, 6.2, 0.6, 766.3, 1.4, 5.7, 1),
    ("Apple,Raw,W/Skin", 81.4, 9.7, 0.2, 73.1, 3.7, 21, 0.3),
    ("Grapes", 15.1, 3.4, 0.1, 24, 0.2, 4.1, 0.2),
    ("Chocolate Chip Cookies", 78.1, 6.2, 0.4, 101.8, 0, 9.3, 0.9),
    ("Lowfat Milk", 121.2, 296.7, 0.1, 500.2, 0, 11.7, 8.1),
    ("Raisin Brn", 115.1, 12.9, 16.8, 1250.2, 4, 27.9, 4),
    ("Hotdog", 242.1, 23.5, 2.3, 0, 0, 18, 10.4)
]

In [4]:
Food = namedtuple("Food", ["name", "unit_cost", "qmin", "qmax"])
Nutrient = namedtuple("Nutrient", ["name", "qmin", "qmax"])

In [32]:
def number_of_foods(mdl_, sol):
    quantity_food = mdl_.find_matching_vars(pattern="q_")
    return sum(1 for qf in quantity_food if sol[qf] >= 1e-5)

def build_diet_model(mdl, **kwargs):
    ints = kwargs.pop('ints', False)
    
    # Create tuples with named foods and nutrients
    foods = [Food(*f) for f in FOODS]
    nutrients = [Nutrient(*n) for n in NUTRIENTS]
    food_nutrients = {(fn[0], nutrients[n].name):
                         fn[1 + n] for fn in FOOD_NUTRIENTS for n in range(len(NUTRIENTS))}
    
    # Decision variables, limited to be >= Food.qmin and <= Food.qmax
    qty = mdl.integer_var_dict(foods, 
                               lb = lambda f: f.qmin, ub = lambda f: f.qmax, 
                               name = lambda f: "q_{}".format(f.name))
    # Limit range of nutrients
    for nut in nutrients:
        amount_nutrients = mdl.sum(qty[f] * food_nutrients[f.name, nut.name] for f in foods)
        mdl.add_range(nut.qmin, 
                      amount_nutrients,
                      nut.qmax)
        mdl.add_kpi(amount_nutrients, publish_name = "Total quantity of {}".format(nut.name))
    
    # Objective function (minimize)
    total_cost = mdl.sum(qty[f] * f.unit_cost for f in foods)
    
    mdl.add_kpi(number_of_foods)
    mdl.minimize(total_cost)
    
    return mdl

#-----
# Solve the model and display the result
#-----

if __name__== '__main__':
    with Model(name='diet', log_output=True, float_precision=6) as mdl:
        build_diet_model(mdl, ints=True)
        mdl.print_information()
        solution = mdl.solve()
        if solution:
            print("------------------")
            solution.display()
        else:
            print("model has no solution")

Model: diet
 - number of variables: 9
   - binary=0, integer=9, continuous=0
 - number of constraints: 7
   - linear=7
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP
Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 0 rows and 1 columns.
MIP Presolve modified 1 coefficients.
Reduced MIP has 7 rows, 15 columns, and 63 nonzeros.
Reduced MIP has 1 binaries, 8 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Found incumbent of value 6.840000 after 0.01 sec. (0.09 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Reduced MIP has 7 rows, 15 columns, and 63 nonzeros.
Reduced MIP has 1 binaries, 8 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, 